In [ ]:
!pip install plotly
!pip install spotipy

# Data 101: Choosing your next festival w/ Data

Today we will be looking at a simple end to end pipeline for extracting, transforming and analyzing data. Our starting point are summer festivals. We will leverage Spotify's API to extract **audio features** from the top tracks of headliners across 25 different European festivals in 2022.

Our work will be divided in 3 steps:

1. **Data Extraction**: loading the festival headliner data & connecting to the Spotify API;
1. **Data Mashing**: use our extracted data to characterize each festival according to the musical characteristics of featured artists;
1. **Data Analysis**: visualize our dataset and use ML to choose a festival according to our favorite artists!

**Data Sources**
1. Festival Headliners: https://www.festicket.com/magazine/discover/top-20-music-festivals-Europe/
1. Spotify API: https://developer.spotify.com/documentation/web-api/

# Data Extraction

In this step we will load our festival headliner data and use it to extract **audio features** from each artists Top 10 tracks using the Spotify API.

## Festival/Headliner Table
Let's start loading our festival table. This data was manually scraped from *Festicket's* list of top 20 festivals in Europe for 2022. You can find the extracted data inside Google Sheets:

https://docs.google.com/spreadsheets/d/1aUiwstZKEENiw3KAT1CCCXcji1iLWdCL_HwngvLxzJc/edit?usp=sharing

We will use Pandas, a Python library for representing and manipulating tabular data, to load our spreadsheet.

In [ ]:
import pandas as pd


We can use a simple renaming trick to read a Google Sheet's spreadsheet through it's sharing URL:

In [ ]:
def read_from_gsheets(spreadsheet_link):
    """
    Transform Google Sheets URL into a CSV file
    """
    working_spreadsheet = spreadsheet_link.replace(
        "/edit?usp=sharing", "/export?format=csv"
    )

    return pd.read_csv(working_spreadsheet)


Now, let's use our function to read the first Festival data table:

In [ ]:
tb_festivals = read_from_gsheets(
    "https://docs.google.com/spreadsheets/d/1aUiwstZKEENiw3KAT1CCCXcji1iLWdCL_HwngvLxzJc/edit?usp=sharing"
)
tb_festivals.head()


We can see this table contains each festival's line-up. Now we will see how we can use the Spotify API to enrich this data.

## Spotify API

The Spotify API is a way to connect to Spotify and extract publicly available data. It's a very rich source of musical data, allowing us to extract information from specific Artits, Albums & Tracks.

We will look at how we can use this resource to enrich our festival data, seeking information for each headliner and their top 10 tracks - but first let's see how we can connect to the Spotify API.

In Python, we can use the `spotipy` library - which makes it really easy to connect and get information from Spotify!

https://towardsdatascience.com/discovering-your-music-taste-with-python-and-spotify-api-b51b0d2744d

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


We must connect to Spotify using a `CLIENT_ID` & `CLIENT_SECRET`: this how we *log-on* to Spotify through our script.

To create a `CLIENT_ID`/`CLIENT_SECRET` of your own you have to log-on to https://developer.spotify.com/dashboard/login w/ your own spotify account and create an App.

In [ ]:
CLIENT_ID = "cd9cd1ff2710474993b146e14b5e2612"
CLIENT_SECRET = "6bb783f4e3114cfc88ed7dbbfa9eee08"

auth_manager = SpotifyClientCredentials(
    client_id=CLIENT_ID, client_secret=CLIENT_SECRET
)
spotify = spotipy.Spotify(client_credentials_manager=auth_manager)


Now we can use the `spotify` object to fetch information from the Spotify API! Let's start extracting musical data from a specific track:

1. **Running Up That Hill** by Kate Bush
1. **Raining Blood** by Slayer
1. **You Only Live Twice** by Nancy Sinatra
1. **I Like It** by Cardi B.

### What are Audio Features?

Audio Features are quantitative data we can extract for different tracks through the Spotify API. We have some obvious things such as **Song Duration** or **Beats per Minute (BPM)**, but we also have some quantities which Spotify uses to classify songs:

1. **Danceability**: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.

1. **Energy**: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.

1. **Valence**: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

You can find the full feature set @ https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features.

We can use the `spotify.audio_features()` method to extract this data for a specific song through it's Spotify URL.

In [ ]:
dict_songs = {
    "kate_bush": "https://open.spotify.com/track/75FEaRjZTKLhTrFGsfMUXR?si=26398bff72014b5a",
    "slayer": "https://open.spotify.com/track/4fiOTntQKr24p07FvQDHZE?si=148a03ca6ba844fb",
    "n_sinatra": "https://open.spotify.com/track/790jn9vbIPMRzhH9Ft4Vji?si=58a6771903db4527",
    "cardi_b": "https://open.spotify.com/track/58q2HKrzhC3ozto2nDdN4z?si=6f755b3e29d841ad",
}


In [ ]:
spotify.audio_features(dict_songs["cardi_b"])


Audio Features gives us a way to quantitavely analyze musical data. However, they are an **attribute** of **individual tracks**. Now we must find a way to calculate something similiar for our Festivals!

### Extracting Top Tracks by Artist

Well, we already have which Artist is playing at each festival - we can start by extracting the top tracks from each artist! This way we can use **Audio Features**, an attribute of **individual tracks**, for the top 10 tracks of each artist.

We can use the `spotify.artist_top_tracks()` to extract this information for a specific artist URL:

In [ ]:
dict_artist = {
    "kate_bush": "https://open.spotify.com/artist/1aSxMhuvixZ8h9dK9jIDwL?si=JJfv25h8QXm-3r-j5L1_7g",
    "slayer": "https://open.spotify.com/artist/1IQ2e1buppatiN1bxUVkrk?si=w397LtnvRtm4B3G6JRi9lA",
    "sinatra": "https://open.spotify.com/artist/3IZrrNonYELubLPJmqOci2?si=ZPDqqkWAQQWeDH_0VzWwvQ",
    "cardi_b": "https://open.spotify.com/artist/4kYSro6naA4h99UJvo89HB?si=hfaZb9xCTgW1DEYlSHX0Nw",
}


In [ ]:
top_tracks = spotify.artist_top_tracks(dict_artist["cardi_b"])
top_tracks


This output is much more complex than the audio features! Using APIs usually involves a bit of detective work: we must investigate the information we obtain to understand how we can use it. Let's a bit of API sleuthing then :)

In [ ]:
type(top_tracks)


After a bit of sleuthing we've understood the structure behind the `top_tracks`! We can leverage that structure to extract each top song's `URI` and use this ID to extract the audio features for each top track using a `for` loop:

In [ ]:
for track in top_tracks["tracks"]:
    print(track["name"])
    track_audio_features = spotify.audio_features(track["uri"])[0]
    valence = track_audio_features["valence"]
    danceability = track_audio_features["danceability"]
    energy = track_audio_features["energy"]
    print(f"V: {valence}, D: {danceability}, E: {energy}")


Now we can extract audio features for each of the top 10 tracks by a given artist. We can use this to create a new DataFrame containung audio features for the top 10 tracks of each artist in our Festival Headliner data.

### Putting it all together!

Let's go over the steps we've mapped so far:

1. Extract the Top 10 Songs of each artist in our Festival Headliner data;
1. Extract the Audio Features of each song in the Top 10 Songs.

For this to work properly we must use the Spotify API to extract the *Spotify Link* for each artist in our festival dataset. This is a lot of **boilerplate** coding: more sleuthing to discover how artist objects are structured and how to extract relevante information from the API!

Not only is it *boilerplate*, it takes a while to extract the audio features for all songs (we have ~1400 tracks!).

To keep things in our 1H time-frame, the information extracted in this process was saved as 2 `.csv` files, which we can load and inspect. The code to create these files can be found in the **Appendix** section of this notebook!

First, let's load our **artist information**:

In [ ]:
tb_artist = pd.read_csv("tb_artist.csv")


Now, let's load our **track information**:

In [ ]:
tb_artist_track = pd.read_csv("tb_artist_track.csv")


## Data Mashing

Now that we are done extracting our data, we must find a way to transform the track's Audio Features into Festival characteristics. First, let us recap our data structure so far:

1. **Festivals**: each *Festival* is composed of *Artists*;
1. **Artists**: each *Artist* is composed of multiple *Tracks* and *Artist* attributes;
1. **Tracks**: each *Track* is composed of multiple attributes (our Audio Features).

We can visualize this in the diagram below:

![title](img/data_scheme.png)

Now we will **aggregate** each level of our data structure to the level above: **Tracks** to **Artist** and **Artist** to **Festival**!

### Aggregating **Tracks**

Let's investigate our tracks table, `tb_artist_track`:

In [ ]:
tb_artist_track.head()


Since we have the artists name, we can the `tb_artist_track.groupby()` method to aggregate our track data for each artist. Let's start by calculating the average `energy` of each artist:

In [ ]:
tb_artist_track.groupby("headliners")["energy"].mean()


We can even use the `.sort_values()` method to sort our table from most energetic to least energetic artist:

In [ ]:
tb_artist_track.groupby("headliners")["energy"].mean().sort_values(ascending=False)


Let's use the `.groupby()` method to aggregate the most interesting Audio Features by artist:

1. Energy
1. Danceability
1. Valence
1. Instrumentalness
1. Loudness
1. Tempo(in BPM)
1. Duration (in Minutes)

In [ ]:
interesting_vars = [
    "energy",
    "danceability",
    "valence",
    "instrumentalness",
    "loudness",
    "tempo",
    "duration_min",
]
tb_artist_track.groupby("headliners")[interesting_vars].mean()


Now let's save the results to a new table: `tb_agg_tracks`. 

In [ ]:
tb_agg_tracks = (
    tb_artist_track.groupby("headliners")[interesting_vars].mean().reset_index()
)
tb_agg_tracks.head()


### Joining Aggregated Tracks w/ Artists & Festivals

Now that we have aggregated our track information we must join this information with our artist information, found in the `tb_artist` DataFrame.

In [ ]:
tb_artist.head()


In [ ]:
tb_agg_tracks.head()


In [ ]:
tb_artist.merge(tb_agg_tracks, on="headliners")


In [ ]:
tb_artist_enriched = tb_artist.merge(tb_agg_tracks, on="headliners")


This table has the average of the selected Audio Features + the popularity & number of followers for each artist. We can join this new enriched table with our original festival data (`tb_festivals`).

In [ ]:
tb_festivals_enriched = tb_festivals.merge(tb_artist_enriched, on="headliners")
tb_festivals_enriched.head()


### Aggregating by Festival

The final step in our data mashing is aggregating our `tb_festivals_enriched`: in this table we have the Audio Features of each headliner playing in a festival.

We can use the same method we did for aggregating **Tracks** to **Artist**: `tb_festivals_enriched.groupby()`! Let's calculate the average Audio Features for each festival:

In [ ]:
tb_festivals_enriched.groupby("festival").mean()


Let's save the results to a new DataFrame: `tb_agg_festival`.

In [ ]:
tb_agg_festival = tb_festivals_enriched.groupby("festival").mean().reset_index()
tb_agg_festival.head()


## Data Analysis & Modelling

Now we arrive at the end-goal of every data collection/transformation pipeline: analysis! We will focus on creating graphs understand the characteristics of each festival according to the Audio Features of that festival's headliner.

### Visualizing our Data

First  we must import our plotting library. For this webinar we will be using **Plotly** (https://plotly.com). Plotly allows us to build interactive graphs inside our Notebook.

In [ ]:
import plotly.express as px


#### Festivals by Artist Popularity

In [ ]:
px.bar(
    data_frame=tb_agg_festival.sort_values("popularity"), x="festival", y="popularity"
)



#### Festivals by Audio Features
##### Audio Feature Distribution

In [ ]:
px.histogram(data_frame=tb_agg_festival, x="duration_min")


#### Comparing Features

In [ ]:
px.scatter(
    data_frame=tb_agg_festival,
    x="popularity",
    y="duration_min",
    hover_name="festival",
    height=600,
    width=600,
)


In [ ]:
px.scatter(
    data_frame=tb_agg_festival,
    x="danceability",
    y="valence",
    hover_name="festival",
    height=600,
    width=600,
)


##### Classifying Features

Sometimes it's easier to visualize numeric data by transforming it into categories. Let's use the function `pd.qcut` to create a classification for the 3 main Audio Features: energy, danceability & valence.

In [ ]:
pd.qcut(tb_agg_festival["valence"], 3, labels=["Sad", "Unsure", "Happy"])


In [ ]:
tb_agg_festival["classif_valence"] = pd.qcut(
    tb_agg_festival["valence"], 3, labels=["Sad", "Unsure", "Happy"]
)
tb_agg_festival["classif_valence"].value_counts()


In [ ]:
tb_agg_festival["classif_energy"] = pd.qcut(
    tb_agg_festival["energy"], 3, labels=["Low", "Mild", "Wild"]
)
tb_agg_festival["classif_dance"] = pd.qcut(
    tb_agg_festival["danceability"], 3, labels=["Low", "Mild", "Dancefloor"]
)


In [ ]:
px.sunburst(
    tb_agg_festival,
    path=["classif_energy", "classif_dance", "classif_valence", "festival"]
)


In [ ]:
px.sunburst(
    tb_agg_festival,
    path=["classif_energy", "classif_dance", "classif_valence"], 
    color='popularity',
    color_continuous_scale='RdBu'
)


 # Appendix

 Although we covered a lot of ground today, some code was too complex to go over during the webinar. A lot of it is **boilerplate** code: code which has no data-centric functionality, like enriching our Festival Headliner data with Spotify URLs.

 The code used to create `tb_artist` and `tb_artist_tracks` can be found below

 ## Extracting Artist Information

In [ ]:
artist_info = []
for artist in tb_festivals["headliners"].unique():
    try:
        artist_dict = {}
        artist_dict["headliners"] = artist
        search_result = spotify.search(q=artist, type="artist")
        artist_dict["uri"] = search_result["artists"]["items"][0]["uri"]
        artist_dict["popularity"] = search_result["artists"]["items"][0]["popularity"]
        artist_dict["followers"] = search_result["artists"]["items"][0]["followers"][
            "total"
        ]
        artist_info.append(artist_dict)
    except IndexError:
        print(f"{artist} not found")

tb_artist = pd.DataFrame(artist_info)


In [ ]:
tb_artist.to_csv("tb_artist.csv", index=False)


## Extracting Top 10 Tracks Information

In [ ]:
track_info = []
for artist_uri in tb_artist["uri"].unique():
    artist = (
        tb_artist.loc[tb_artist["uri"] == artist_uri, :]
        .reset_index()
        .loc[0, "headliners"]
    )
    top_10_tracks = spotify.artist_top_tracks(artist_uri)["tracks"]
    for track in top_10_tracks:
        track_data = spotify.audio_features(track["uri"])[0]
        track_data["headliners"] = artist
        track_data["artist_uri"] = artist_uri
        track_info.append(track_data)

tb_artist_track = pd.DataFrame(track_info)


In [ ]:
tb_artist_track["duration_min"] = tb_artist_track["duration_ms"] / (60 * 1000)
tb_artist_track.to_csv("tb_artist_track.csv", index=False)
